In [20]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import t

df = pd.read_csv('Temp_MN.csv')

# Povprečje in standardni odklon
mu = df['TEMP'].mean()
sigma = df['TEMP'].std(ddof=1) # <- nepristranska cenilka

# 95% napovedni interval za naslednji dan
#Ker sigma ni znan, moramo uporabiti Studentovo porazdelitev, vendar se pri 1114 prostostnih
# stopnjah kvantili Studentove porazdelitve že zelo ujemajo s kvantili N(0, 1).
# Ker privzemamo neodvisnost temperatur med dnevi, dobimo napovedni interval za temperaturo naslednji dan.
sp_meja = mu - 1.96 * sigma #/ np.sqrt(1115) Pri napovednem intervalu ne deliš
zg_meja = mu + 1.96 * sigma #/ np.sqrt(1115) Pri napovednem intervalu ne deliš
interval = (sp_meja, zg_meja)
print(f"povprečje: {mu}")
print(f"standardni odklon: {sigma}")
print(f"95% napovedni interval (beli šum): {interval}")

povprečje: 14.090421524663677
standardni odklon: 12.910692967851453
95% napovedni interval (beli šum): (np.float64(-11.21453669232517), np.float64(39.395379741652526))


In [21]:
from scipy.optimize import curve_fit
import numpy as np

df = pd.read_csv('Temp_MN.csv')

# Pretvori v dan v letu (1-365)
dan_v_letu = ((df['DAN'] - 1) % 365) + 1

def sezonski_model(t, a0, a1, a2):
    return a0 + a1 * np.sin(2 * np.pi * t / 365) + a2 * np.cos(2 * np.pi * t / 365)

# Prileganje modela
# popt ... optimalni parametri
# pcov ... kovariančna matrika
popt, pcov = curve_fit(sezonski_model, dan_v_letu, df['TEMP'])
ostanki = df['TEMP'] - sezonski_model(dan_v_letu, *popt)
sigma = ostanki.std(ddof=1)

# Napoved za naslednji dan (zadnji dan + 1)
zadnji_dan = df['DAN'].iloc[-1]
naslednji_dan = zadnji_dan + 1
naslednji_dan_v_letu = ((naslednji_dan - 1) % 365) + 1
napoved = sezonski_model(naslednji_dan_v_letu, *popt)


# 95% napovedni interval za naslednji dan
#Ker sigma ni znan, moramo uporabiti Studentovo porazdelitev, vendar se pri 1114 prostostnih
# stopnjah kvantili Studentove porazdelitve že zelo ujemajo s kvantili N(0, 1).
# Ker privzemamo neodvisnost Šumov med dnevi, dobimo napovedni interval za temperaturo naslednji dan.
sp_meja1 = napoved - 1.96 * sigma #/ np.sqrt(1115) Pri napovednem intervalu ne deliš. Dodaj vir!
zg_meja1 = napoved + 1.96 * sigma #/ np.sqrt(1115) Pri napovednem intervalu ne deliš. Dodaj vir!
interval = (sp_meja1, zg_meja1)
print(f"Napoved za dan {naslednji_dan} (dan v letu {naslednji_dan_v_letu}): {napoved:.2f}")
print(f"95% napovedni interval (sezonski model): {interval}")

Napoved za dan 1195 (dan v letu 100): 17.47
95% napovedni interval (sezonski model): (np.float64(6.1126773786819655), np.float64(28.823965026782332))


In [22]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import norm

# Naloži podatke
df = pd.read_csv('Temp_MN.csv')

# Poskrbi, da so podatki razvrščeni po dnevu
df = df.sort_values('DAN')

# Izračunaj spremembe temperatur med zaporednimi dnevi (delta_T)
df['delta_T'] = df['TEMP'].diff()
df['delta_DAN'] = df['DAN'].diff()

# Uporabi le tiste vrstice, kjer je razlika dneva 1 (zaporedni dnevi)
mask = df['delta_DAN'] == 1
delta_T = df.loc[mask, 'delta_T']
dan_v_letu = ((df.loc[mask, 'DAN'] - 1) % 365) + 1

# Sezonski model za spremembe
def sezonska_diferenca(t, b1, b2):
    return b1 * np.sin(2 * np.pi * t / 365) + b2 * np.cos(2 * np.pi * t / 365)

# Prileganje modela
popt, pcov = curve_fit(sezonska_diferenca, dan_v_letu, delta_T)
ostanki = delta_T - sezonska_diferenca(dan_v_letu, *popt)
sigma = ostanki.std(ddof=1)

# Napoved za naslednji dan (1195)
naslednji_dan = 1195
naslednji_dan_v_letu = ((naslednji_dan - 1) % 365) + 1
delta_pred = sezonska_diferenca(naslednji_dan_v_letu, *popt)
T_pred = df['TEMP'].iloc[-1] + delta_pred

# 95% napovedni interval za naslednji dan
#Ker sigma ni znan, moramo uporabiti Studentovo porazdelitev, vendar se pri 1114 prostostnih
# stopnjah kvantili Studentove porazdelitve že zelo ujemajo s kvantili N(0, 1).
# Ker privzemamo neodvisnost sprememb temperatur med dnevi, dobimo napovedni interval za temperaturo naslednji dan.
sp_meja2 = T_pred - 1.96 * sigma #/ np.sqrt(1115) Pri napovednem intervalu ne deliš
zg_meja2 = T_pred + 1.96 * sigma #/ np.sqrt(1115) Pri napovednem intervalu ne deliš
interval = (sp_meja2, zg_meja2)
print(f"Napoved za dan {naslednji_dan} (dan v letu {naslednji_dan_v_letu}): {T_pred:.2f}")
print(f"95% napovedni interval (model sprememb): {interval}")

Napoved za dan 1195 (dan v letu 100): 8.03
95% napovedni interval (model sprememb): (np.float64(-0.7310227021261806), np.float64(16.794053979070114))
